# Engine: Database Connection - Snowflake 

### Overview: 

Because Howso Engine is an API-first platform built upon Pandas DataFrames, it is very easy to connect to most databases. This includes Amazon AWS, Google GCP, Microsoft Azure Studios, and Snowflake. 

In this notebook, we will adapt the [1-engine-intro.ipynb](https://github.com/howsoai/howso-engine-recipes/blob/main/1-engine-intro.ipynb) recipe to read-in a dataset from Snowflake. For information on your specific data warehouse, please consult your data vendor's documentation.

### Recipe Goals:

This notebook will provide a demonstration of how to load a table from Snowflake. To follow along with the recipe, please download the [Adult dataset](https://github.com/howsoai/howso-engine-recipes/blob/main/data/adult/adult/adult.tsv.gz) and upload it to your Snowflake account. The exact details of how to do this are beyond the scope of this recipe, but there are numerous resources online.

**Note**: This notebook assumes a user has a source dataset within Snowflake and have all the credentials needed access it. They should also be familiar with basic SQL statements.   

Once the dataset is uploaded, install the [Snowflake Python/Pandas connector](https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-pandas#requirements).

In [1]:
# Activate the correct environment and then:
%pip install "snowflake-connector-python[pandas]"

In [2]:
import pandas as pd
from pathlib import Path
from pmlb import fetch_data
from IPython.display import Markdown

from howso.engine import Trainee, load_trainee
from howso.utilities import infer_feature_attributes

## Step 1: Import Data

Connecting to a database is akin to scanning a badge to access your office. It requires all the correct credentials. For security reasons, your sensitive information is typically not supposed to be exposed in the code. In this example, the private information is masked and it is implied you must input your own credentials to access your dataset. Please consult your database or IT administrator for your organization's procedures.


In [4]:
import snowflake.connector

# establish connection
ctx = snowflake.connector.connect(
    user=YOUR_user_information,
    password=YOUR_password_information,
    account=YOUR_account_information,
    warehouse=YOUR_warehouse,
    database=YOUR_database,
    schema=YOUR_schema,
)
cs = ctx.cursor()

# retrieve the table (dataset)
source_table_name = "adult"

try:
    sql = f'''
        SELECT *
        FROM "{source_table_name}"
        ORDER BY RANDOM()
        LIMIT 2000
        '''
    cs.execute(sql)
    df = cs.fetch_pandas_all()
    display(Markdown(f'### "{source_table_name.capitalize()}" table:'))
    display(df)

# close connection to database
finally:
    cs.close()
ctx.close()

### "Adult" table:

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,25.0,6,136450.0,7,12.0,2,1,5,4,0,0.0,0.0,35.0,39,0
1,23.0,4,130905.0,9,13.0,4,12,3,4,0,0.0,0.0,40.0,39,1
2,59.0,4,268840.0,15,10.0,2,1,2,4,0,0.0,0.0,16.0,39,0
3,28.0,4,79702.0,15,10.0,4,1,2,2,1,0.0,0.0,40.0,39,1
4,29.0,4,40295.0,11,9.0,2,10,0,4,1,0.0,0.0,40.0,39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,47.0,4,192835.0,11,9.0,2,1,0,4,1,0.0,0.0,50.0,39,0
1996,50.0,4,175804.0,11,9.0,2,12,0,4,1,0.0,0.0,48.0,39,1
1997,50.0,4,82566.0,12,14.0,2,13,0,4,1,0.0,0.0,45.0,39,0
1998,34.0,4,157165.0,15,10.0,0,8,4,4,0,0.0,0.0,40.0,39,1


Our example dataset for this recipe is the well known `Adult` dataset. This dataset consists of 14 Context Features and 1 Action Feature. The Action Feature in this version of the `Adult` dataset has been renamed to `target` and it takes the form of a binary indicator for whether a person in the data makes more than $50,000/year (*target*=1) or less (*target*=0).

## Step 2: Feature Mapping

Typically, an exploratory analysis is done on the data to get a general feel of the descriptive statistics and data attributes. 

Methods like `describe` from a Pandas dataframe often automatically present these types of information of interest to a user, as shown below. While informative, these descriptive statistics are often used as a sanity check pre- and post-modeling and a model typically doesn't actually use any of these feature attributes.

In [5]:
df.describe()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000
mean,38.615000,3.862000,189210.028500,10.273000,10.158000,2.636000,6.466500,1.467500,3.652000,0.672000,1314.236000,97.251500,40.34200,36.782000,0.756500
std,13.801812,1.478529,106386.888184,3.721881,2.597578,1.516461,4.218504,1.608178,0.856308,0.469602,8699.654387,429.418206,12.15092,7.756582,0.429302
min,17.000000,0.000000,19302.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.00000,0.000000,0.000000
25%,28.000000,4.000000,116983.250000,9.000000,9.000000,2.000000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000,40.00000,39.000000,1.000000
50%,37.000000,4.000000,174946.500000,11.000000,10.000000,2.000000,6.000000,1.000000,4.000000,1.000000,0.000000,0.000000,40.00000,39.000000,1.000000
75%,48.000000,4.000000,239621.000000,12.000000,13.000000,4.000000,10.000000,3.000000,4.000000,1.000000,0.000000,0.000000,45.00000,39.000000,1.000000
max,90.000000,8.000000,860348.000000,15.000000,16.000000,6.000000,14.000000,5.000000,4.000000,1.000000,99999.000000,3175.000000,99.00000,41.000000,1.000000


In the Howso Engine workflow, feature attributes are an essential part of model building and usage. By incorporating certain feature attributes into training process itself, Howso Engine gains another layer of information that will help in fine-tuning the results. 

In order to assist the user with defining the feature attributes, Howso has an `infer_feature_attributes` tool that automatically processes the dataset for the user.

In Howso Engine, these feature attributes are model infrastructure feature parameters that are based on the existing data, rather than exact descriptive statistics. This is why, for example, the min and max bounds on continuous features are not the exact min and max values of the dataset, but rather an expanded version of those min and max values to allow for some variation.

In [6]:
# Infer features attributes
features = infer_feature_attributes(df)
features

{'age': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 7.38905609893065, 'max': 148.4131591025766}},
 'workclass': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 1},
  'bounds': {'allow_null': False}},
 'fnlwgt': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 8103.083927575384, 'max': 1202604.2841647768}},
 'education': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 1},
  'bounds': {'allow_null': False}},
 'education-num': {'type': 'continuous',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 1.0, 'max': 20.085536923187668}},
 'marital-status': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': '

In [7]:
# Specify Context and Action Features
action_features = ['target']
context_features = features.get_names(without=action_features)


> **Note:** Train Test Split

To gauge model performance, train-test splits are often used in traditional machine learning workflows. Howso Engine does not require the use of train-test split for validation. Please see recipe `6-validation.ipynb` for further explanation. Therefore, we will not use train-test splits our recipes unless the test set serves a specific purpose.

## Step 3: Create Trainee

To begin the Howso Engine workflow, a Trainee is created to act as a base for all of our ML needs. In all subsequent notebooks and jupyter cells, we will refer to Howso Engine's model as Trainee.

**Definitions:**

**`Trainee`:** A collection of Cases that comprise knowledge. May include metadata and parameters. In traditional ML this is referred to as a model.

**`Cases`:** A set of feature values representing a situation observed.  In traditional ML, a Case is sometimes referred to as an "observation", "record", or "data point". In database terms, a Case would be a row of values. For supervised learning a Case is a set of Context Features and Action Features and for unsupervised learning a Case is just a set of features. 

In [8]:
# Create the Trainee
t = Trainee(
    features=features,
    overwrite_existing=True
)

Version 9.1.2 of Howso Engine™ is available. You are using version 7.1.0.


## Step 4: Preprocessing and Training

One benefit of Howso Engine is that most standard forms of data pre-processing such as one-hot encoding and standardization are **NOT** needed, which is in contrast to many traditional ML models. This does not include more sophisticated forms of pre-processing such as feature selection or feature engineering, which may still be useful. Fitting is also done in two steps in Howso Engine.

**Definitions:**

**`Train`:** Exposing a Trainee to a Case which may cause the ML algorithm to update the Trainee. This is a single training step; training may happen at each decision, at a certain sampling rate of observations per second, or at certain events.

**`Analyze`:** Tune internal parameters to improve performance and accuracy of predictions and metrics. Analysis may be targeted or targetless.  Targetless analysis provides the best balanced set of hyperparameters if an Action Feature is not specified, along with a performance boost while targeted analysis provides a boost to accuracy. 


In [9]:
# Train
t.train(df)

# Analyze the Trainee
t.analyze(context_features=context_features, action_features=action_features)

## Step 5: Results

Once Howso Engine is trained and analyzed, it provides the user with a variety of ML capabilities. At this stage in the Howso Engine workflow, a typical use case would be to evaluate the accuracy of the Trainee, which is performed by the `react` method. This is equivalent to `predict` in many traditional Machine Learning workflows, although the `react` method is not solely used for supervised predictions as detailed in subsequent recipes. 

Since we are not using a train-test split, we will use the `react_into_trainee` method, which performs a `react` on each of the cases that is trained into the model. Alternatively, a standard `react` call may be used on unseen data for prediction.
The accuracy is calculated internally as shown in the code below and this is the recommended accuracy metric. Further explanations are available in recipe **6-validation**.

**Definitions:**

**`React`:** Exposing a Trainee to a new Case's Context Features and an Action Feature for that case is returned. In traditional ML this is often referred to predicting or labeling.

In [10]:
# Recommended metrics
t.react_into_trainee(action_feature=action_features[0], residuals=True)
stats = t.get_prediction_stats().target.round(2)
print(f'Howso Prediction Results - Accuracy: {stats["accuracy"]}, Precision: {stats["recall"]}, and Recall: {stats["precision"]}')

Howso Prediction Results - Accuracy: 0.83, Precision: 0.77, and Recall: 0.75


In [11]:
stats_matrix = t.get_prediction_stats(stats=["confusion_matrix"])
print("Howso Prediction Results - Confusion Matrix")
matrix = pd.DataFrame(stats_matrix.loc["confusion_matrix", "target"])
matrix.index.name = "Predicted"
matrix.columns.name = "Actual"
display(matrix)

Howso Prediction Results - Confusion Matrix


Actual,0,1
Predicted,,
0,137,69
1,99,695


Another way to use the `get_prediction_stats` method is to utilize the `condition` parameter. Specifying a condition allows the user to get prediction stats on the subset of cases that matches the specified condition.

For example, we can use the `condition` parameter to find the accuracy, precision, and recall on the subsets of the dataset with each value for Sex.

In [12]:
sex_1_stats = t.get_prediction_stats(condition={'sex': 1})['target'].round(2)
print(f'Howso Prediction Results On Cases with Sex=1 - Accuracy: {sex_1_stats["accuracy"]}, Precision: {sex_1_stats["recall"]}, and Recall: {sex_1_stats["precision"]}')

sex_0_stats = t.get_prediction_stats(condition={'sex': 0})['target'].round(2)
print(f'Howso Prediction Results On Cases with Sex=0 - Accuracy: {sex_0_stats["accuracy"]}, Precision: {sex_0_stats["recall"]}, and Recall: {sex_0_stats["precision"]}')

Howso Prediction Results On Cases with Sex=1 - Accuracy: 0.8, Precision: 0.77, and Recall: 0.75
Howso Prediction Results On Cases with Sex=0 - Accuracy: 0.93, Precision: 0.85, and Recall: 0.67


## Step 6: Saving, Loading, and Deleting from your local disk

These methods work for `HowsoDirectClient` and its subclasses.

### Saving and Loading

When saving to disk, a filepath or filename may provided. The filepath provided may be a relative or absolute path. The filepath can include the desired filename with a `.caml` extension. If no filename is at the end of the filepath, the Trainee ID will be used. If just the filename is provided, then the current working directory will be used as the filepath. 

> **Note:** It is recommended, however not necessary that the filename match the trainee ID


### Deleting

When deleting a Trainee, the `delete` method deletes the trainee from the last saved or loaded disk location, as well as memory. If the Trainee has not been saved, `delete` can also be used to just remove a Trainee from memory.

When Trainees are saved, a `.caml` file and a `.txt` version file is saved. The `delete` method assumes that the prefix to the version filename is the same as the `.caml` trainee filename.

In [13]:
from howso.direct import HowsoDirectClient

# Clients that extend `HowsoDirectClient` will have local file operations
if isinstance(t.client, HowsoDirectClient):
    # Example filepath (same as default)
    cwd = Path.cwd()
    file_path=f"{cwd}/engine_intro_trainee.caml"

    # Saving
    t.save(file_path=file_path)

    # Loading
    t = load_trainee(file_path=file_path)

    # Deleting
    t.delete()

## Conclusion and Next Steps

Databases are ubiquititous in the era of "Big Data," and it is imperative solutions can connect to an organization's data warehouse. By being an API-first platform, Howso offers the freedom and flexibility to connect to many of the popular cloud warehouses. 

Use this recipe as a template for connecting to your data, and ultimately, building your use-cases!